Necessary imports

In [2]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.5/725.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15/15 [openai]14/15 [openai]c]-types]


In [3]:
from openai import OpenAI

In [ ]:
client = OpenAI()

First, lift with Foxdec. Call LLM to convert to Cheri

In [9]:
wc_file = open("wc.asm")

In [10]:
prompt = "You are an expert in compilers. Translate this NASM code into LLVM IR for CHERI-RISCV architecture, full code only, no explanation: " + wc_file.read()
print(prompt)

You are an expert in compilers. Translate this NASM code into LLVM IR for CHERI-RISCV architecture, full code only, no explanation: extern _ITM_deregisterTMCloneTable
extern _ITM_registerTMCloneTable
extern __ctype_b_loc
extern __cxa_finalize
extern __gmon_start__
extern __libc_start_main
extern __stack_chk_fail
extern exit
extern fclose
extern feof
extern fopen
extern fputc
extern getc
extern perror
extern printf
extern stderr
extern vfprintf


global _IO_stdin_used
global __TMC_END__
global __bss_start
global __data_start
global __dso_handle
global _edata
global _end
global _fini
global _init
global _start
global ccount
global counter
global getword
global lcount
global main
global report
global stderr@GLIBC_2.2.5
global total_ccount
global total_lcount
global total_wcount
global wcount


default rel


; ----------------
; Function: _start
; ----------------
; Entry 11a0; block 0; address 11a0
_start:
  XOR EBP, EBP
  MOV R9, RDX
  POP RSI
  MOV RDX, RSP
  AND RSP, 0xfffffffffffffff0

In [11]:
response = client.responses.create(model = "gpt-4.1", input=prompt)

In [12]:
print(response.output_text)

```llvm
target datalayout = "e-m:e-p:256:256-i64:64-i128:128-n64-S128"
target triple = "riscv64cheri-unknown-unknown"

@__dso_handle = global ptr @__dso_handle, align 8
@__TMC_END__ = global [0 x i8] zeroinitializer, align 8
@stderr = external global ptr
@L_.bss_0x4028 = global i8 0, align 8
@L_.bss_0x4030 = global i64 0, align 8
@L_.bss_0x4038 = global i64 0, align 8
@L_.bss_0x4040 = global i64 0, align 8
@L_.bss_0x4048 = global i64 0, align 8
@L_.bss_0x4050 = global i64 0, align 8
@L_.bss_0x4058 = global i64 0, align 8

@L_.rodata_0x2004 = private unnamed_addr constant [2 x i8] c" \00", align 1
@L_.rodata_0x2006 = private unnamed_addr constant [22 x i8] c"%6lu %6lu %6lu %s\0A\00", align 1
@L_.rodata_0x2019 = private unnamed_addr constant [2 x i8] c"r\00", align 1
@L_.rodata_0x201b = private unnamed_addr constant [21 x i8] c"cannot open file `%s'\00", align 1
@L_.rodata_0x2031 = private unnamed_addr constant [22 x i8] c"usage: wc FILE [FILE...]\00", align 1
@L_.rodata_0x204a = private

Need to find a way to check if this code:

1) Compiles for Cheri

2) Is the same as the given .asm file

In [15]:
with open("wc.ll", "w") as text_file:
    text_file.write(response.output_text[7:])

In [18]:
# Repeat compile with errors generated
import subprocess

In [ ]:
success = False
threshold = 10
current = 1

while not success and  current < threshold:
    result = subprocess.run(['llc','wc.ll'], stdout=subprocess.PIPE)
    #print(result.stdout.decode("utf-8"))
    if result.stdout.__contains__(b'error'):
        prompt = "You are an expert in compilers. The foollowing CHERI-RISCV LLVM IR .ll code produces an error when compiled using the llc command.\n The code is: " + wc_file.read() +"\n The error is: " + result.stdout.decode("utf-8")
        print(prompt)
        success = False
        current = current + 1
    else:
        success = True

False


llc: error: unable to get target for 'riscv64cheri-unknown-unknown', see --version and --triple.

b''


llc: error: unable to get target for 'riscv64cheri-unknown-unknown', see --version and --triple.

In [16]:
!llc wc.ll

llc: error: unable to get target for 'riscv64cheri-unknown-unknown', see --version and --triple.